In [21]:
from splinter import Browser
import sqlalchemy
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import time
from sqlalchemy import create_engine
import sqlite3

browser = Browser('chrome')
url = 'https://www.pickleheads.com/courts/us/florida'
browser.visit(url)
buttons = browser.find_by_css(".css-2ttlle")
button_index_to_click = 1
if button_index_to_click < len(buttons):
    button_to_click = buttons[button_index_to_click]
    button_to_click.click()
    time.sleep(2)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [22]:
whole_list=soup.find('div',class_='css-12eoos1')

In [18]:
location = []
city = []
courts=[]

for entry in whole_list:
    location_data = entry.find('span', class_='css-uyrttv')
    if location_data:
        location.append(location_data.text)
    else:
        continue

    city_data = entry.find(class_='css-1vl3ez2')
    
    if city_data:
        city.append(city_data.text)

    court_data=entry.select('.css-1vl3ez2')
    if court_data: 
        last_span=court_data[-1] 
        courts.append(last_span.text)

1033


In [19]:
#Create a dataframe with the locations and cities.
pickleball_dict={"Location Name":location,
                 "City":city,
                 "Number of Courts":courts}
fl_pickleball_df=pd.DataFrame(pickleball_dict)
fl_pickleball_df_sorted=fl_pickleball_df.sort_values(by='City',ascending=True)
city_county_df=pd.read_csv('Florida City County and Coordinates.csv')
city_county_df=city_county_df.rename(columns={"CITY":"City","COUNTY":"County","LATITUDE":"Latitude","LONGITUDE":"Longitude"})

In [24]:
fl_full_info=pd.merge(fl_pickleball_df_sorted,city_county_df,on='City',how='left')
fl_full_info.to_csv('pickleball_courts_with_counties_updated.csv',index=False)
fl_full_info['Number of Courts'] = fl_full_info['Number of Courts'].str.replace(' courts', '')
fl_full_info['Number of Courts'] = fl_full_info['Number of Courts'].str.replace(' court', '')
fl_pickleball_file = 'court_location.sqlite'
conn = sqlite3.connect(fl_pickleball_file)
fl_full_info.to_sql('county_courts', conn, if_exists='replace', index=False)

1033

In [ ]:
browser.quit()